# Applied Data Science Capstone


## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### For this assignment, we will explore and cluster the neighborhoods in Toronto.

We will use the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

First we will import the required libraries for this project

In [1]:
# Import all libraries and dependencies

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # intall geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes #install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Data preparation

What I have done was to save the Toronto-FST table present in the Wikipedia "List of postal codes of Canada: M" ( https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M ), to a csv file and i have stored this file on IBM Cloud storage bucket. The file I have named Toronto-FSAs.csv

In [2]:
# Load the dataset from IBM Cloud

!wget -q -O 'Toronto-FSAs.csv' https://cos-standard-n78-neighborhoods-in-toronto.s3.us-south.cloud-object-storage.appdomain.cloud/Toronto-FSAs.csv
print('Data downloaded!')

# load the table to pandas dataframe
toronto_neighborhoods_data = pd.read_csv('Toronto-FSAs.csv')

#  inspect the 5 first items
toronto_neighborhoods_data.head(5)

Data downloaded!


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The dataframe is not yet ready to be explored, since there are few "Borough" = "Not assigned", I need to clean the dataframe and remove those "Not assigned"

### Req_01: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
# Remove all rows from dataframe where Borough = 'Not assigned'
#df = df[condition]
#df. drop(df[condition]. index, axis=0, inplace=True)
#toronto_neighborhoods_data = toronto_neighborhoods_data['Borough' == "Not assigned"]

index_names = toronto_neighborhoods_data[toronto_neighborhoods_data['Borough'] == 'Not assigned'].index

toronto_neighborhoods_data.drop(index_names, axis=0, inplace=True)

toronto_neighborhoods_data.head(12)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


We are still not ready, I need to work on the dataframe to fullfil the requirement Req_02

###  Req_02: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
# we are going to use groupby on label 'Postcode' and'Borough', and use aggregate 

# groups_toronto_df = toronto_neighborhoods_data.groupby('Postcode')

df= toronto_neighborhoods_data.groupby(['Postcode', 'Borough']).agg(lambda x: ','.join(x.unique())).reset_index()

df.tail(12)

,Postcode,Borough,Neighborhood
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
93,M9A,Queen's Park,Not assigned
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."


In [10]:
df.head(14)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


We still need to clean data in the dataframe in order to replace any neighborhood with value 'Not assigned' (e.g. row 93 in the dataframe)

### Req_3: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [11]:
# df['count'] = np.where(df['count'] > 0, 1, df['count'])

df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

df.tail(12)

,Postcode,Borough,Neighborhood
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
93,M9A,Queen's Park,Queen's Park
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."


Now the dataframe is cleaned, and we can go for the next requirement that is:

### to print the number of rows of your dataframe

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [13]:
df.head(14)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In order to get the coordinates (Latitude and Logitude), I have download the CSV provided by Coursera and uploade in IBM Cloud storage bucket. we will get the data from there and read to a new dataframe and after we will merge both dataframes.

In [14]:
# URL from the Bucket https://cos-standard-n78-neighborhoods-in-toronto.s3.us-south.cloud-object-storage.appdomain.cloud/Geospatial_Coordinates.csv

# Load the dataset from IBM Cloud

!wget -q -O 'Geospatial_Coordinates.csv' https://cos-standard-n78-neighborhoods-in-toronto.s3.us-south.cloud-object-storage.appdomain.cloud/Geospatial_Coordinates.csv
print('Data downloaded!')

# load the table to pandas dataframe
toronto_geospatial_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

#  inspect the 5 first items
toronto_geospatial_coordinates.head(5)

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# We are going to merge both dataframes in order to have Postalcode | Borough | Neighborhood | Latitude | Longitude

toronto_merge_df = pd.merge(df, toronto_geospatial_coordinates, left_on='Postcode', right_on='Postal Code')

toronto_merge_df.head(12)

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


In [19]:
toronto_merge_df.tail(12)

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",M8Y,43.636258,-79.498509
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",M8Z,43.628841,-79.520999
93,M9A,Queen's Park,Queen's Park,M9A,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",M9B,43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",M9C,43.643515,-79.577201
96,M9L,North York,Humber Summit,M9L,43.756303,-79.565963
97,M9M,North York,"Emery,Humberlea",M9M,43.724766,-79.532242
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",M9R,43.688905,-79.554724


In [20]:
toronto_merge_df.shape

(103, 6)

Now, we are going to drop the column "Postal Code" from toronto_merge_df, in order to have a dataframe as requested by the requirements.

In [22]:
toronto_df = toronto_merge_df

toronto_df.drop(['Postal Code'], axis=1, inplace=True)

toronto_df.tail(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery,Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724


In [23]:
toronto_df.shape

(103, 5)

## Explore and cluster the neighborhoods in Toronto. 

Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [24]:
address = 'Toronto,Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### let's simplify the above map and segment and cluster only the neighborhoods in Etobicoke. So let's slice the original dataframe and create a new dataframe of the Etobicoke data.

In [26]:
etobicoke_data = toronto_df[toronto_df['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
4,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999


Let's get the geographical coordinates of Etobicoke.

In [27]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.671459150000004, -79.55249206611668.


#### As we did with all of Toronto, let's visualize Etobicoke the neighborhoods in it.

In [28]:
# create map of Manhattan using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [29]:
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
CLIENT_ID = 'HALLXBXGZGHCYB5VMP1PPSQ43D2TJWGU5ORSDY4OZDWQKW5K' # your Foursquare ID
CLIENT_SECRET = 'T5IRXH5ZXUCISPPEI1UI0RW02KISPDOFUIWZBKPK2II53PJL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HALLXBXGZGHCYB5VMP1PPSQ43D2TJWGU5ORSDY4OZDWQKW5K
CLIENT_SECRET:T5IRXH5ZXUCISPPEI1UI0RW02KISPDOFUIWZBKPK2II53PJL


#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [30]:
etobicoke_data.loc[0, 'Neighborhood']

'Humber Bay Shores,Mimico South,New Toronto'

#### Get the neighborhood's latitude and longitude values.

In [31]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humber Bay Shores,Mimico South,New Toronto are 43.6056466, -79.50132070000001.


### Now, let's get the top 100 venues that are in Humber Bay Shores within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HALLXBXGZGHCYB5VMP1PPSQ43D2TJWGU5ORSDY4OZDWQKW5K&client_secret=T5IRXH5ZXUCISPPEI1UI0RW02KISPDOFUIWZBKPK2II53PJL&v=20180605&ll=43.6056466,-79.50132070000001&radius=500&limit=100'

Send the GET request and examine the results:

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1930af77af03001b129010'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.6101466045, 'lng': -79.49511771930959},
   'sw': {'lat': 43.6011465955, 'lng': -79.50752368069043}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b119977f964a520488023e3',
       'name': 'LCBO',
       'location': {'address': '2762 Lake Shore Blvd W',
        'crossStreet': 'btwn 1st & 2nd St',
        'lat': 43.60228082768786,
        'lng': -79.4993016827402,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we are ready to clean the json and structure it into a pandas dataframe.

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.602281,-79.499302
1,Domino's Pizza,Pizza Place,43.601583,-79.500905
2,New Toronto Fish & Chips,Restaurant,43.601849,-79.503281
3,Delicia Bakery & Pastry,Bakery,43.601403,-79.503012
4,Lucky Dice Restaurant,Café,43.601392,-79.503056


And how many venues were returned by Foursquare?

In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Etobicoke

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### CreateEtobicoke venues dataframe

In [38]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Humber Bay Shores,Mimico South,New Toronto
Alderwood,Long Branch
The Kingsway,Montgomery Road,Old Mill North
Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea
Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Westmount
Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Northwest


##### Let's check the size of the resulting dataframe

In [39]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(69, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
2,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
3,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


##### Let's check how many venues were returned for each neighborhood

In [40]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe",8,8,8,8,8,8
"Humber Bay Shores,Mimico South,New Toronto",10,10,10,10,10,10
"Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",1,1,1,1,1,1
"Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips",4,4,4,4,4,4
"Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor",13,13,13,13,13,13
Northwest,3,3,3,3,3,3
"The Kingsway,Montgomery Road,Old Mill North",3,3,3,3,3,3


### Let's find out how many unique categories can be curated from all the returned venues

In [41]:
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 37 uniques categories.


### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Thrift / Vintage Store,Video Store,Wings Joint
0,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,"Humber Bay Shores,Mimico South,New Toronto",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Thrift / Vintage Store,Video Store,Wings Joint
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.0,0.100,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.200000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.100000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000
1,"Alderwood,Long Branch",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.111111,0.222222,0.111111,0.111111,0.000000,0.0,0.000000,0.111111,0.111111,0.000000,0.000000,0.0,0.000000
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.00,0.125,0.000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.125,0.000,0.00,0.125000,0.125,0.125000,0.125000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Humber Bay Shores,Mimico South,New Toronto",0.100000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.100,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.000000,0.100000,0.000000,0.000,0.100,0.000,0.00,0.000000,0.100,0.100000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Humber Bay,King's Mill Park,Kingsway Park Sout...",0.000000,0.000000,1.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Kingsview Village,Martin Grove Gardens,Richvie...",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.25,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.25,0.000000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Kingsway Park South West,Mimico NW,The Queensw...",0.076923,0.000000,0.0,0.000,0.076923,0.076923,0.00,0.000,0.000,0.000000,0.076923,0.076923,0.000000,0.076923,0.0,0.076923,0.076923,0.076923,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.076923,0.000000,0.076923,0.076923,0.0,0.076923
7,Northwest,0.000000,0.333333,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"The Kingsway,Montgomery Road,Old Mill North",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.333333,0.000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Westmount,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.125,0.125000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.000,0.125,0.00,0.000000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,0.000000,0.000000,0.0,0.000000


And let's examine the new dataframe size.

In [45]:
etobicoke_onehot.shape

(69, 38)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [46]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Thrift / Vintage Store,Video Store,Wings Joint
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.0,0.100,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.200000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.100000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000
1,"Alderwood,Long Branch",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.111111,0.222222,0.111111,0.111111,0.000000,0.0,0.000000,0.111111,0.111111,0.000000,0.000000,0.0,0.000000
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.00,0.125,0.000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.125,0.000,0.00,0.125000,0.125,0.125000,0.125000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Humber Bay Shores,Mimico South,New Toronto",0.100000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.100,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.000000,0.100000,0.000000,0.000,0.100,0.000,0.00,0.000000,0.100,0.100000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Humber Bay,King's Mill Park,Kingsway Park Sout...",0.000000,0.000000,1.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Kingsview Village,Martin Grove Gardens,Richvie...",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.25,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.25,0.000000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Kingsway Park South West,Mimico NW,The Queensw...",0.076923,0.000000,0.0,0.000,0.076923,0.076923,0.00,0.000,0.000,0.000000,0.076923,0.076923,0.000000,0.076923,0.0,0.076923,0.076923,0.076923,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.076923,0.000000,0.076923,0.076923,0.0,0.076923
7,Northwest,0.000000,0.333333,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"The Kingsway,Montgomery Road,Old Mill North",0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.00,0.333333,0.000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Westmount,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.00,0.000,0.125,0.125000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.000,0.125,0.00,0.000000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,0.000000,0.000000,0.0,0.000000


#### Let's confirm the new size

In [47]:
etobicoke_grouped.shape

(10, 38)

#### Let's print each neighborhood along with the top 5 most common venues

In [48]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
           venue  freq
0    Pizza Place   0.2
1  Grocery Store   0.2
2    Video Store   0.1
3     Beer Store   0.1
4       Pharmacy   0.1


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1   Coffee Shop  0.11
2           Pub  0.11
3           Gym  0.11
4  Skating Rink  0.11


----Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe----
          venue  freq
0  Liquor Store  0.12
1          Park  0.12
2    Beer Store  0.12
3     Pet Store  0.12
4      Pharmacy  0.12


----Humber Bay Shores,Mimico South,New Toronto----
          venue  freq
0        Bakery   0.1
1  Liquor Store   0.1
2     Pet Store   0.1
3      Pharmacy   0.1
4           Gym   0.1


----Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea----
                 venue  freq
0       Baseball Field   1.0
1    

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Video Store,Fast Food Restaurant,Beer Store,Sandwich Place,Drugstore,Discount Store
1,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Coffee Shop,Pub,Pool,Pharmacy,Café,Drugstore
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Coffee Shop,Pharmacy,Beer Store,Liquor Store,Café,Park,Pet Store,Pizza Place,Fast Food Restaurant,Drugstore
3,"Humber Bay Shores,Mimico South,New Toronto",Bakery,Café,Fast Food Restaurant,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,Gym,Restaurant
4,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Baseball Field,Wings Joint,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Chinese Restaurant


## 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 2, 0, 3, 4, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etobicoke_merged = etobicoke_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,3.0,Bakery,Café,Fast Food Restaurant,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,Gym,Restaurant
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484,3.0,Pizza Place,Gym,Skating Rink,Sandwich Place,Coffee Shop,Pub,Pool,Pharmacy,Café,Drugstore
2,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,1.0,Pool,River,Park,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop
3,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509,2.0,Baseball Field,Wings Joint,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Chinese Restaurant
4,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,3.0,Wings Joint,Hardware Store,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Gym,Bakery
